### This Jupyter notebook demonstrates the creation of a simple neural network for educational purposes. The goal is to make the neural network easily interpretable for beginners and help develop a better basic intuition of what complex, thousand-unit multilayer neural networks are doing in a nutshell.



In [86]:
import numpy as np

data_to_classify = np.array([
[
    [1,1,1],
    [1,0,1],
    [1,0,1], # Zero
    [1,0,1],
    [1,1,1],   
],
[
    [0,0,1],
    [0,0,1],
    [0,0,1], # One
    [0,0,1],
    [0,0,1],   
],
[
    [1,1,1],
    [0,0,1],
    [1,1,1], # Two
    [1,0,0],
    [1,1,1],   
],
[
    [1,1,1],
    [0,0,1],
    [1,1,1],  # Three
    [0,0,1],
    [1,1,1],
],
[
    [1,0,1],
    [1,0,1],
    [1,1,1],  # Four
    [0,0,1],
    [0,0,1],
],
[
    [1,1,1],
    [1,0,0],
    [1,1,1],  # Five
    [0,0,1],
    [1,1,1],
],
[
    [1,1,1],
    [1,0,0],
    [1,1,1],  # Six
    [1,0,1],
    [1,1,1],
],
[
    [1,1,1],
    [0,0,1],
    [0,1,1],  # Seven
    [0,0,1],
    [0,0,1],
],
[
    [1,1,1],
    [1,0,1],
    [1,1,1],  # Eight
    [1,0,1],
    [1,1,1],
],
[
    [1,1,1],
    [1,0,1],
    [1,1,1],  # Nine
    [0,0,1],
    [1,1,1],
]
])

input_layer = np.full((5, 3, 2), [1, 0])

# neuron_recognizing_left_edge = [
#     [[1, 0],[0, 0],[0, 0]],
#     [[1, 0],[0, 0],[0, 0]],
#     [[1, 0],[0, 0],[0, 0]],
#     [[1, 0],[0, 0],[0, 0]],
#     [[1, 0],[0, 0],[0, 0]],  
# ]

neuron_recognizing_right_edge = [
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[1, 0]],  
]

# neuron_recognizing_top_edge = [
#     [[1, 0],[1, 0],[1, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],  
# ]

# neuron_recognizing_bottom_edge = [
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[1, 0],[1, 0],[1, 0]],  
# ]

# neuron_recognizing_middle_plank = [
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[1, 0],[1, 0],[1, 0]],
#     [[0, 0],[0, 0],[0, 0]],
#     [[0, 0],[0, 0],[0, 0]],  
# ]

neuron_recognizing_3_horizontal_stripes = [
    [[1, 0],[1, 0],[1, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[1, 0],[1, 0],[1, 0]],
    [[0, 0],[0, 0],[0, 0]],  
    [[1, 0],[1, 0],[1, 0]],  
]

neuron_recognizing_middle_dot = [
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[1, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],  
]

neuron_recognizing_2nd_row_leftmost_pixel = [
    [[0, 0],[0, 0],[0, 0]],
    [[1, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],  
]

neuron_recognizing_2nd_row_rightmost_pixel = [
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],  
]

neuron_recognizing_4th_row_leftmost_pixel = [
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[1, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],  
]

neuron_recognizing_4th_row_rightmost_pixel = [
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[0, 0]],
    [[0, 0],[0, 0],[1, 0]],
    [[0, 0],[0, 0],[0, 0]],  
]


hidden_layer = np.array([
    neuron_recognizing_3_horizontal_stripes,
    neuron_recognizing_middle_dot,
    neuron_recognizing_right_edge,
    neuron_recognizing_2nd_row_leftmost_pixel,
    neuron_recognizing_2nd_row_rightmost_pixel,
    neuron_recognizing_4th_row_leftmost_pixel,
    neuron_recognizing_4th_row_rightmost_pixel,
]);

neuron_classifying_0 = [
    [1, 0], [-100, 0], [0, 0], [1, 0], [1, 0], [1, 0], [1, 0],
]

neuron_classifying_1 = [
    [-1, 0], [-1000, 0], [2, 0], [-1000, 0], [0, 0], [-1000, 0], [0, 0],
]

neuron_classifying_2 = [
    [1, 0], [0, 0], [0, 0], [-100, 0], [0, 0], [0, 0], [-100, 0],
]

neuron_classifying_3 = [
    [1, 0], [0, 0], [0, 0], [-100, 0], [1, 0], [-100, 0], [1, 0], 
]

neuron_classifying_4 = [
    [0, 0], [1, 0], [1, 0], [3, 0], [0, 0], [-100, 0], [0, 0], 
]

neuron_classifying_5 = [
    [1, 0], [0, 0], [0, 0], [1, 0], [-100, 0], [-100, 0], [1, 0], 
]

neuron_classifying_6 = [
    [1, 0], [0, 0], [0, 0], [1, 0], [-100, 0], [2, -1], [1, 0], 
]

neuron_classifying_7 = [
    [0, 0], [9, 0], [0, 0], [-100, 0], [0, 0], [-100, 0], [0, 0], 
]

neuron_classifying_8 = [
    [1, 0], [0, 0], [0, 0], [1, 0], [1, 0], [2, -1], [1, 0],
]

neuron_classifying_9 = [
    [1, 0], [0, 0], [0, 0], [1, 0], [1, 0], [-100, 0], [1, 0],
]

output_layer = np.array([
    neuron_classifying_0,
    neuron_classifying_1,
    neuron_classifying_2,
    neuron_classifying_3,
    neuron_classifying_4,
    neuron_classifying_5,
    neuron_classifying_6,
    neuron_classifying_7,
    neuron_classifying_8,
    neuron_classifying_9
])

# def compute_activation(input_data, layer):
#     """
#     Computes the activation values for the next layer in the pseudo neural network.
#     """
#     # Special case for input layer
#     if len(layer.shape) == 3:  # Input layer has shape (5, 3, 2)
#         # Transform input data into pairs [value, 0]
#         result = np.zeros((5, 3, 2))
#         result[:,:,0] = input_data  # Set first value of each pair to input value
#         return result
    
#     # For other layers
#     activation_values = []
#     for neuron in layer:
#         activation = 0
#         for i in range(len(input_data)):
#             for j in range(len(input_data[i])):
#                 activation += input_data[i][j] * neuron[i][j][0]
#                 activation += neuron[i][j][1]
#         activation_values.append(activation)
#     return activation_values


def relu(x):
    return max(0, x)

def compute_activation(input_data, layer):
    """
    Computes the activation values for the next layer in the pseudo neural network.
    """
    # Convert input_data to numpy array if it isn't already
    input_data = np.array(input_data)
    
    # Case 1: Input layer processing (input is 5x3 matrix)
    if len(layer.shape) == 3 and len(input_data.shape) == 2:  
        result = np.zeros((5, 3, 2))
        result[:,:,0] = input_data
        return result
    
    # Case 2: Hidden layer processing (input is 5x3x2 matrix)
    elif len(layer.shape) == 4:  # Hidden layer has shape (5, 5, 3, 2)
        activation_values = []
        for neuron in layer:
            activation = 0
            for i in range(len(input_data)):
                for j in range(len(input_data[i])):
                    activation += input_data[i][j][0] * neuron[i][j][0]
                    activation += neuron[i][j][1]
                    activation = relu(activation)
            activation_values.append(activation)
        return activation_values
    
    # Case 3: Output layer processing (input is array of 5 values)
    else:  # Output layer processing
        if len(input_data.shape) == 1:  # If input is 1D array
            activation_values = []
            for neuron in layer:
                activation = 0
                for i in range(len(input_data)):
                    activation += input_data[i] * neuron[i][0]
                    activation += neuron[i][1]
                    activation = relu(activation)
                activation_values.append(activation)
            return activation_values

    raise ValueError(f"Unexpected input shape {input_data.shape} or layer shape {layer.shape}")

neuron_layers = {
    "input layer": input_layer,
    "hidden layer": hidden_layer,
    "output layer": output_layer,
}

zero = data_to_classify[5]

input_for_hidden_layer = compute_activation(zero, neuron_layers["input layer"])
input_for_output_layer = compute_activation(input_for_hidden_layer, neuron_layers["hidden layer"])
output_layer_result = compute_activation(input_for_output_layer, neuron_layers["output layer"])

def output_probabilities(output_layer_result):
    # output_layer_result_pos = list(map(lambda x: x + abs(min(output_layer_result)), output_layer_result))
    # output_layer_result_prob = list(map(lambda x: x ))
    min_max_normalized_result = list(map(lambda x: ((x - min(output_layer_result)) / (max(output_layer_result) - min(output_layer_result))), output_layer_result))
    print(min_max_normalized_result)

print(output_layer_result)
# output_probabilities(output_layer_result)

zero = data_to_classify[6]

input_for_hidden_layer = compute_activation(zero, neuron_layers["input layer"])
input_for_output_layer = compute_activation(input_for_hidden_layer, neuron_layers["hidden layer"])
output_layer_result = compute_activation(input_for_output_layer, neuron_layers["output layer"])


output_probabilities(output_layer_result)


[2.0, 0, 0, 1.0, 8.0, 11.0, 10.0, 0, 10.0, 11.0]
[0.25, 0.0, 0.0, 0.08333333333333333, 0.0, 0.08333333333333333, 1.0, 0.0, 1.0, 0.08333333333333333]
